# Task 3: Heart Disease Prediction (Fixed & Final)

### 🎯 Objective
Predict whether a patient is at risk of heart disease using health-related features.
This version is tailored for a dataset where:
- `sex` is 'Male'/'Female'
- Booleans are stored as 'TRUE'/'FALSE'
- `cp`, `thal`, `slope` etc. may be categorical
- Target column is `num` (we'll map it to binary)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

%matplotlib inline

In [ ]:
# Load dataset (ensure file is present)
df = pd.read_csv('heart_disease_uci.csv')
df.head()

In [ ]:
# Drop unnecessary columns and fill NaNs
df.drop(columns=[col for col in ['id', 'dataset'] if col in df.columns], inplace=True)
df = df.fillna(0)

In [ ]:
# Convert booleans
for col in ['fbs', 'exang']:
    if col in df.columns:
        df[col] = df[col].map({'TRUE': 1, 'FALSE': 0})

# Convert sex
if 'sex' in df.columns and df['sex'].dtype == object:
    df['sex'] = df['sex'].map({'Male': 1, 'Female': 0})

In [ ]:
# Convert target
if 'num' in df.columns:
    df['target'] = df['num'].apply(lambda x: 1 if x > 0 else 0)
    df.drop(columns=['num'], inplace=True)

In [ ]:
# Safe one-hot encoding
for col in ['cp', 'restecg', 'slope', 'thal']:
    if col in df.columns:
        df = pd.get_dummies(df, columns=[col], drop_first=True)

In [ ]:
# Feature-target split
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Logistic Regression
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)
y_pred_log = log_model.predict(X_test)

# Decision Tree
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)

In [ ]:
# Accuracy
print('Logistic Regression Accuracy:', accuracy_score(y_test, y_pred_log))
print('Decision Tree Accuracy:', accuracy_score(y_test, y_pred_tree))

In [ ]:
# Confusion matrix
sns.heatmap(confusion_matrix(y_test, y_pred_log), annot=True, fmt='d', cmap='Blues')
plt.title('Logistic Regression - Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# ROC Curve
y_prob_log = log_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_prob_log)
roc_auc = roc_auc_score(y_test, y_prob_log)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Logistic Regression (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True)
plt.show()

### ✅ Conclusion
- Dataset cleaned and all required fields processed.
- Boolean values and categoricals handled safely.
- Trained two classifiers: Logistic Regression and Decision Tree.
- Evaluated using accuracy, confusion matrix, and ROC curve.
